#### 5% Training for pre-initalization 

In [1]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train_top = np.load('x_3000_bottomview.npy')
y_train_top = np.load('3000_y.npy')

x_train = x_train_top
y_train = y_train_top


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (3000, 28, 28, 1)
3000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3000 samples, validate on 10000 samples
Epoch 1/12
3000/3000 [==============================] - 8s 3ms/step - loss: 1.9851 - acc: 0.2863 - val_loss: 1.3724 - val_acc: 0.5164
Epoch 2/12
3000/3000 [==============================] - 4s 1ms/step - loss: 1.2627 - acc: 0.5513 - val_loss: 1.0698 - val_acc: 0.6084
Epoch 3/12
3000/3000 [==============================] - 4s 1ms/step - loss: 0.9176 - acc: 0.6687 - val_loss: 0.7453 - val_acc: 0.7073
Epoch 4/12
3000/3000 [==============================] - 4s 1ms/step - loss: 0.7393 - acc: 0.7340 - val_loss: 0.7526 - val_acc: 0.7160
Epoch 5/12
3000/3000 [==============================] - 4s 1ms/step - loss: 0.6536 - acc: 0.7657 - val_loss: 

In [2]:
model.save('bottomView.h5') 

#### 95% for fining tune on top of pre-initailized model

In [3]:
from keras.models import load_model
model = load_model('bottomView.h5')

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.load('57000_x.npy')
y_train = np.load('57000_y.npy')

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


model = load_model('bottomView.h5')

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (57000, 28, 28, 1)
57000 train samples
10000 test samples
Train on 57000 samples, validate on 10000 samples
Epoch 1/12
57000/57000 [==============================] - 38s 666us/step - loss: 0.1639 - acc: 0.9504 - val_loss: 0.0553 - val_acc: 0.9829
Epoch 2/12
57000/57000 [==============================] - 37s 646us/step - loss: 0.0819 - acc: 0.9749 - val_loss: 0.0338 - val_acc: 0.9887
Epoch 3/12
57000/57000 [==============================] - 37s 654us/step - loss: 0.0640 - acc: 0.9814 - val_loss: 0.0367 - val_acc: 0.9878
Epoch 4/12
57000/57000 [==============================] - 38s 669us/step - loss: 0.0505 - acc: 0.9845 - val_loss: 0.0273 - val_acc: 0.9916
Epoch 5/12
57000/57000 [==============================] - 42s 731us/step - loss: 0.0450 - acc: 0.9869 - val_loss: 0.0204 - val_acc: 0.9939
Epoch 6/12
57000/57000 [==============================] - 46s 811us/step - loss: 0.0394 - acc: 0.9879 - val_loss: 0.0258 - val_acc: 0.9928
Epoch 7/12
57000/57000 [===================

In [5]:
pred = model.predict(x_test)
print(pred.shape)

(10000, 10)


In [6]:
np.save('bottomview_pred.npy',pred)